## Differential Geometry of Vector Manifolds
Emphasis on the curvature tensor
1. extringsic geometry
2. behavior under transformation
3. generalization to nonReimannian curvature
4. unification of intrinsic and extrinsic geometry
5. regard the curl, rather than the curvature as the fundamental object in differential geometry

But shouldn't the intrinsic geometry employ only tensors with values in the tangent algebra?

In [1]:
#1.3 ellipsoid
# Let M be an ellipsoid in R3, (ax)^{2}+(by)^{2}+z^{2}=d
# x = 1/a*cos(p)*sin(t)*e1 + 1/b*sin(p)*sin(t)*e2 + cos(t)*e3, t, p in R

from gc_utils import *
alg = Algebra(3)
locals().update(alg.blades)

c2e = lambda p, t: 4*np.cos(p)*np.sin(t)*e1 + 5*np.sin(p)*np.sin(t)*e2 + np.cos(t)*e3
e_p = lambda p, t: -4*np.sin(p)*np.sin(t)*e1 + 5*np.cos(p)*np.sin(t)*e2
e_t = lambda p, t: (4*np.cos(p)*np.cos(t)*e1) + (5*np.sin(p)*np.cos(t)*e2) - (np.sin(t)*e3)
def Ix(x):
    p = np.arctan((4/5)*x.e2/x.e1)
    t = np.arccos(x.e3)
    return e_p(p, t) ^ e_t(p, t)

# |p| < pi/2, 0 < t < pi for unique arctan & arccos
p, t = -1, 3
x = c2e(p, t)
I = Ix(x)
nIx = lambda x: normalize(Ix(x))
nI = nIx(x)

nI, nI ^ (3*e_t(p,t) + 5*e_p(p,t))

(1.0 𝐞₁₂ + -0.024 𝐞₁₃ + -0.0192 𝐞₂₃, 1.29e-15 𝐞₁₂₃)

### Curl and Curvature


In [2]:
C, D, E = [random_multivector(alg) for _ in range(3)]
V1, V2 = [random_versor(2, alg) for _ in range(2)]
f = lambda x: V1.sw(inv(x)) + V2.sw(x**3)
A = lambda x: E*x**2 + C*x*D
a = lambda x: A(x).grade(1)
PA = lambda x: P(A(x), Ix(x))
pa = lambda x: P(a(x), Ix(x))
f_ = lambda a: lambda x: differential(f, x, a)

In [ ]:
# Recall these: 
# the shape operator
def shape(Ix, alg):
    return lambda x, A: derivative(lambda x: P(A, Ix(x)), x, alg, Ix=Ix)

# the curl tensor
curl_tensor = lambda a, x: curl(lambda x: P(a, Ix(x)), x, alg, Ix=Ix)

# the spur
N = lambda x: sum(o(lambda x: P(r, Ix(x))) for r, o in derivative_gen(x, alg, Ix=Ix))

In [107]:
# The curl tensor Sa is the angular velocity of I as it slides along the manifold in the direction a
Sa = curl_tensor(pa(x), x)
differential(nIx, x, pa(x)), nI.cp(Sa)

(8.03e-05 𝐞₁₂ + 0.0347 𝐞₁₃ + -0.0391 𝐞₂₃,
 8.03e-05 𝐞₁₂ + 0.0347 𝐞₁₃ + -0.0391 𝐞₂₃)

In [153]:
#1.2 see 4.3.13
frame = blade_split(I, alg)
r_frame = reciprocal(frame)
B = lambda x: C*x**2 + E*x*D
b = lambda x: B(x).grade(1)
pb = lambda x: P(b(x), Ix(x))
codiff_a = lambda A: lambda y: codiff(A, y, a(x), Ix, h=1e-3) # a(x) as a fixed direction
codiff_b = lambda A: lambda y: codiff(A, y, b(x), Ix, h=1e-3)
def operator_cp(f, g):
    return lambda A: lambda x: f(g(A))(x) - g(f(A))(x)

a_ = a(x)
b_ = b(x)

(
    operator_cp(codiff_a, codiff_b)(A)(x),
    sum(o1(lambda x: sum(((b_^a_)|(r1^r))* o(A) for r, o in coderivative_gen(x, alg, Ix=Ix))) for r1, o1 in coderivative_gen(x, alg, Ix=Ix, h=1e-3))
    )

(-0.00301 + -0.0446 𝐞₁ + -0.0118 𝐞₂ + -0.000574 𝐞₃ + -0.0145 𝐞₁₂ + 0.000349 𝐞₁₃ + 0.00028 𝐞₂₃,
 -0.00301 + -0.0446 𝐞₁ + -0.0118 𝐞₂ + -0.000574 𝐞₃ + -0.0145 𝐞₁₂ + 0.000349 𝐞₁₃ + 0.00028 𝐞₂₃)

In [140]:
#1.2 Observe the subltle difference between codifferential and directional coderivative
lie = lie_bracket(a, b, alg, Ix)
codiff_a_ = lambda A: lambda y: codiff(A, y, a(y), Ix, h=1e-3) # a(y) as a coordinate line
codiff_b_ = lambda A: lambda y: codiff(A, y, b(y), Ix, h=1e-3)
(
    operator_cp(codiff_a_, codiff_b_)(A)(x) - codiff(A, x, lie(x), Ix, h=1e-3), 
    operator_cp(codiff_a_, codiff_b_)(A)(x)
    )

(-0.00301 + -0.0446 𝐞₁ + -0.0118 𝐞₂ + -0.000574 𝐞₃ + -0.0145 𝐞₁₂ + 0.000349 𝐞₁₃ + 0.00028 𝐞₂₃,
 -0.258 + -0.0989 𝐞₁ + 1.5 𝐞₂ + -0.0378 𝐞₃ + 1.66 𝐞₁₂ + -0.0399 𝐞₁₃ + -0.032 𝐞₂₃)

In [166]:
#1.3
PA = lambda x: P(A(x), Ix(x))
diff_a = lambda F: lambda x: differential(F, x, pa(x))
diff_b = lambda F: lambda x: differential(F, x, pb(x))
pb_ = pb(x)
pa_ = pa(x)
Pb = projection_differential(Ix, pb)
Pa = projection_differential(Ix, pa)
Sb = curl_tensor(pb(x), x)
(
    codiff_b_(codiff_a_(PA))(x), 
    P(diff_b(diff_a(PA))(x), I) + Pb(Pa(PA))(x),
    P(diff_b(diff_a(PA))(x), I) + P(Sb.cp(A(x).cp(Sa)), I),
    )

(1.71 + 0.41 𝐞₁ + -0.568 𝐞₂ + 0.0215 𝐞₃ + -0.626 𝐞₁₂ + 0.015 𝐞₁₃ + 0.0121 𝐞₂₃,
 1.71 + 0.41 𝐞₁ + -0.568 𝐞₂ + 0.0215 𝐞₃ + -0.626 𝐞₁₂ + 0.015 𝐞₁₃ + 0.0121 𝐞₂₃,
 1.71 + 0.41 𝐞₁ + -0.568 𝐞₂ + 0.0215 𝐞₃ + -0.626 𝐞₁₂ + 0.015 𝐞₁₃ + 0.0121 𝐞₂₃)

In [ ]:
#1.3 first eq from this and apply 4.2.11
diff_a(PA)(x), Pa(PA)(x) + P(diff_a(PA)(x), I)

(0.486 + -0.116 𝐞₁ + 0.566 𝐞₂ + 0.0383 𝐞₃ + 0.453 𝐞₁₂ + -0.00582 𝐞₁₃ + -0.0144 𝐞₂₃,
 0.486 + -0.116 𝐞₁ + 0.566 𝐞₂ + 0.0383 𝐞₃ + 0.453 𝐞₁₂ + -0.00582 𝐞₁₃ + -0.0144 𝐞₂₃)

In [ ]:
#1.4
codiff_a = lambda A: lambda y: codiff(A, y, pa(x), Ix, h=1e-3) # a(x) as a fixed direction
codiff_b = lambda A: lambda y: codiff(A, y, pb(x), Ix, h=1e-3)
(
    operator_cp(codiff_a, codiff_b)(PA)(x), 
    operator_cp(Pa, Pb)(PA)(x), 
    PA(x).cp(P(Sa.cp(Sb), I))
    )

(-2.53e-08 + -0.00185 𝐞₁ + 0.00134 𝐞₂ + -6.78e-05 𝐞₃ + 1.06e-07 𝐞₁₂ + -2.54e-09 𝐞₁₃ + -2.04e-09 𝐞₂₃,
 -1.26e-26 + -0.00185 𝐞₁ + 0.00134 𝐞₂ + -6.78e-05 𝐞₃ + -4.76e-14 𝐞₁₂ + 7.91e-12 𝐞₁₃ + -1.23e-11 𝐞₂₃,
 -0.00185 𝐞₁ + 0.00134 𝐞₂ + -6.78e-05 𝐞₃ + 8.47e-22 𝐞₁₃)

In [ ]:
#1.5
Sb_ = lambda x: curl_tensor(pb(x), x)
(
    P(Sa.cp(Sb), I),
    Pa(Sb_)(x), #4.2.34
    
    )

(0.00216 𝐞₁₂ + -5.18e-05 𝐞₁₃ + -4.15e-05 𝐞₂₃,
 0.00216 𝐞₁₂ + -5.18e-05 𝐞₁₃ + -4.15e-05 𝐞₂₃)

In [5]:
# The curvature tensor


Intrinsic geometry:
1. Unaware of deformations that preserve distances
2. sum of the angles in a triangle exceed pi -> how curved the space
3. Gaussian curvature

Extrinsic geometry:
1. mean curvature and second fundamental form
2. Plane vs. cylinder

Connection:

Gauss-Codazzi equations:

The second fundamental form (extrinsic curvature) is constrained by the first fundamental form (intrinsic metric)

The Gauss equation relates the Riemann curvature (intrinsic) to the shape operator (extrinsic).

The Codazzi equations describe how the second fundamental form varies across the surface.